In [93]:
import openpyxl

key_file = openpyxl.open("auth/weather_key.xlsx")
key_dict = {}

excel_sheet = key_file.active
for key in excel_sheet.rows:
    key_dict[key[0].value] = key[1].value

key_file.close()

In [159]:
from collections import Counter

def get_max_iscs(iscs): 
    iscs_list = re.findall(r'\{([^}]+)', iscs)
    iscs_counter = Counter(iscs_list)
    max_iscs = ""
    max_iscs_value = 0

    for key in iscs_counter:
        if "강도" in key:
            del_key.append(key)
        else:
            if max_iscs_value < iscs_counter[key]:
                max_iscs = key
                max_iscs_value = iscs_counter[key]

    for key in del_key:
        del iscs_counter[key]

    return max_iscs

In [167]:
from urllib.parse import urlencode
import requests as req
import pprint as pp
import pandas as pd
import re
from datetime import datetime as dt

init_page_no = 1
query_dict = dict({
    "serviceKey" : key_dict['decoding-key'],
    "numOfRows" : 10,
    "pageNo": init_page_no,
    "dataCd": "ASOS",
    "dateCd": "DAY",
    "startDt": "20180501",
    "endDt": "20190430",
    "stnIds": "108",
    "dataType": "JSON"
})    

api_url = "http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList?"
datas = []
while True:
    if (len(datas) % 50) == 0:
        print("Page: {} / Count: {}".format(query_dict['pageNo'], len(datas)))
    query = urlencode(query_dict)
    res = req.get(api_url + query)
    body = res.json()['response']['body']
    
    items = body['items']['item']
    page_info = dict({
        "num_of_rows" : body['numOfRows'],
        "page_no" : body['pageNo'],
        "total_count" : body['totalCount']
    })
    for item in items:
        in_data = dict({
            "날짜": dt.strptime(item['tm'], '%Y-%m-%d'),
            "날씨": "특이사항 없음" if item['iscs'] == "" else get_max_iscs(item['iscs']),
            "평균온도": item['avgTa'],
            "평균습도": item['avgRhm']
        })
        datas.append(in_data)
    
    if len(datas) >= page_info['total_count']:
        break
    else:
        query_dict['pageNo'] = (body['pageNo'] + 1)

Page: 1 / Count: 0
Page: 6 / Count: 50
Page: 11 / Count: 100
Page: 16 / Count: 150
Page: 21 / Count: 200
Page: 26 / Count: 250
Page: 31 / Count: 300
Page: 36 / Count: 350


In [171]:
df_datas = pd.DataFrame(datas)
df_datas

,날짜,날씨,평균온도,평균습도
0,2018-05-01,연무,20.4,72.8
1,2018-05-02,비,15.1,90.4
2,2018-05-03,비,11.2,62.4
3,2018-05-04,특이사항 없음,14.1,45.5
4,2018-05-05,특이사항 없음,18.3,46.8
...,...,...,...,...
360,2019-04-26,비,8.4,82.1
361,2019-04-27,햇무리,11.6,52.6
362,2019-04-28,비,12.9,48.8
363,2019-04-29,햇무리,13.0,52.9


In [182]:
from pymongo import MongoClient as mc

mongo_uri = "mongodb://localhost:27017"
client = mc(mongo_uri)

weather_col = client.keti_pattern_recognition.weather_info

for idx in df_datas.index:
    in_dict = dict({
        "date": df_datas.iloc[idx]['날짜'],
        "weather": df_datas.iloc[idx]['날씨'],
        "avgTa": df_datas.iloc[idx]['평균온도'],
        "avgRhm": df_datas.iloc[idx]['평균습도']
    })
    weather_col.insert_one(in_dict)

client.close()